In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
df = pd.read_excel("SP100_vol.xlsx", index_col="Date",parse_dates=True, sheet_name="SP100 Original")

In [3]:
df.head()

,AMT,COP,CVX,AXP,BAC,C,CB,CME,GS,JPM,...,TMO,UNH,AMZN,HD,LOW,MCD,NKE,SBUX,TGT,TJX
Date,,,,,,,,,,,,,,,,,,,,,
2007-06-27,0.014124,0.018373,0.015768,0.012207,0.010291,0.015468,0.009848,0.014583,0.017107,0.013571,...,0.015377,0.013590,0.014877,0.011604,0.014249,0.008126,0.020990,0.012522,0.011658,0.013830
2007-06-28,0.015762,0.011954,0.011587,0.010778,0.011098,0.013562,0.009857,0.010666,0.014501,0.012888,...,0.011744,0.010659,0.017926,0.009785,0.010404,0.010648,0.015595,0.014401,0.009406,0.012848
2007-06-29,0.011944,0.014456,0.012986,0.013216,0.008582,0.012972,0.010098,0.012094,0.014643,0.013615,...,0.011072,0.015759,0.011421,0.014842,0.012574,0.011361,0.017018,0.014760,0.008964,0.013771
2007-07-02,0.009732,0.009799,0.008922,0.009994,0.006183,0.010277,0.006879,0.009909,0.010316,0.008367,...,0.009402,0.008731,0.012612,0.008851,0.007254,0.007875,0.012682,0.011086,0.009220,0.010368
2007-07-06,0.009956,0.007930,0.007381,0.009133,0.005680,0.007148,0.009542,0.018029,0.009381,0.007802,...,0.009832,0.008183,0.010760,0.009840,0.010354,0.007864,0.010880,0.008781,0.020747,0.011182


In [4]:
train=df.iloc[:int(len(df)*0.8)]

In [5]:
test=df.iloc[int(len(df)*0.8)-22:]

In [6]:
df.iloc[int(len(df)*0.8)]

AMT     0.006287
COP     0.011492
CVX     0.007431
AXP     0.006722
BAC     0.007536
          ...   
MCD     0.004273
NKE     0.009057
SBUX    0.007809
TGT     0.007861
TJX     0.007815
Name: 2018-08-30 00:00:00, Length: 80, dtype: float64

In [7]:
train=np.array(train)

In [8]:
train.shape

(2727, 80)

In [9]:
RV_d = []
for i in range(len(train)-1):
  RV_d.append(train[i].astype(float))

RV_w = []
for i in range(len(train)-5):
  RV_w.append(np.mean(train[i:i+5]).astype(float))

RV_m = []
for i in range(len(train)-22):
  RV_m.append(np.mean(train[i:i+22]).astype(float))

RV_d=RV_d[len(RV_d)-len(RV_m):]
RV_w=RV_w[len(RV_w)-len(RV_m):]

In [10]:
RV_D = np.zeros([len(RV_d),train.shape[1]])
for i in range(len(RV_d)):
  RV_D[i,:]=RV_d[i]

RV_W = np.zeros([len(RV_w),train.shape[1]])
for i in range(len(RV_w)):
  RV_W[i,:]=RV_w[i]

RV_M = np.zeros([len(RV_m),train.shape[1]])
for i in range(len(RV_m)):
  RV_M[i,:]=RV_m[i]

In [11]:
RV_actuals=train[22:,:]

In [12]:
RV_actuals.shape

(2705, 80)

In [13]:
test=np.array(test)

In [14]:
test.shape

(704, 80)

In [15]:
  for i in range(len(test)-1):
    RV_d.append(test[i].astype(float))

  RV_w = []
  for i in range(len(test)-5):
    RV_w.append(np.mean(test[i:i+5]).astype(float))

  RV_m = []
  for i in range(len(test)-22):
    RV_m.append(np.mean(test[i:i+22]).astype(float))

  RV_d=RV_d[len(RV_d)-len(RV_m):]
  RV_w=RV_w[len(RV_w)-len(RV_m):]
  RV_D_test = np.zeros([len(RV_d),test.shape[1]])
  for i in range(len(RV_d)):
    RV_D_test[i,:]=RV_d[i]

  RV_W_test = np.zeros([len(RV_w),test.shape[1]])
  for i in range(len(RV_w)):
    RV_W_test[i,:]=RV_w[i]

  RV_M_test = np.zeros([len(RV_m),test.shape[1]])
  for i in range(len(RV_m)):
    RV_M_test[i,:]=RV_m[i]

In [16]:
forecasts=np.zeros([test.shape[0]-22,test.shape[1]])
for j in range(test.shape[1]):
  Data ={'Actual': RV_actuals[:,j],
  'D': RV_D[:,j],
  'W': RV_W[:,j],
  'M': RV_M[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  rv_scaled = sm.add_constant(rv_scaled)
  rv_scaled['Actual']=rv_scaled['Actual'].astype(float)
  rv_scaled['D']=rv_scaled['D'].astype(float)
  rv_scaled['W']=rv_scaled['W'].astype(float)
  rv_scaled['M']=rv_scaled['M'].astype(float)
  X = rv_scaled.drop("Actual", axis = 1)
  y = rv_scaled["Actual"]
  results = sm.OLS(y, X).fit()
  Data ={'D': RV_D_test[:,j],
  'W': RV_W_test[:,j],
  'M': RV_M_test[:,j]}
  dataframe = pd.DataFrame(data=Data)
  rv_scaled=dataframe
  X_test = sm.add_constant(rv_scaled)
  for i in range(int(len(X_test)/5)):
    forecasts1=(results.params[0]*X_test[X_test.columns[0]][i*5]+results.params[1]*X_test[X_test.columns[1]][i*5]
                  +results.params[2]*X_test[X_test.columns[2]][i*5]+results.params[3]*X_test[X_test.columns[3]][i*5])
    forecasts[i*5,j]=forecasts1
    forecasts2=(results.params[0]*X_test[X_test.columns[0]][i*5]+results.params[1]*forecasts1
                  +results.params[2]*X_test[X_test.columns[2]][i*5]+results.params[3]*X_test[X_test.columns[3]][i*5])
    forecasts[i*5+1,j]=forecasts2
    forecasts3=(results.params[0]*X_test[X_test.columns[0]][i*5]+results.params[1]*forecasts2
                  +results.params[2]*X_test[X_test.columns[2]][i*5]+results.params[3]*X_test[X_test.columns[3]][i*5])
    forecasts[i*5+2,j]=forecasts3
    forecasts4=(results.params[0]*X_test[X_test.columns[0]][i*5]+results.params[1]*forecasts3
                  +results.params[2]*X_test[X_test.columns[2]][i*5]+results.params[3]*X_test[X_test.columns[3]][i*5])
    forecasts[i*5+3,j]=forecasts4
    forecasts5=(results.params[0]*X_test[X_test.columns[0]][i*5]+results.params[1]*forecasts4
                  +results.params[2]*X_test[X_test.columns[2]][i*5]+results.params[3]*X_test[X_test.columns[3]][i*5])
    forecasts[i*5+4,j]=forecasts5

In [17]:
forecasts.shape

(682, 80)

In [18]:
forecasts[0,:]

array([0.00672858, 0.0078891 , 0.00635559, 0.00741461, 0.00741043,
       0.00707343, 0.00531064, 0.00746443, 0.0072442 , 0.00667126,
       0.00815727, 0.00717943, 0.0053662 , 0.00619402, 0.00760916,
       0.00780032, 0.00885232, 0.00581674, 0.00707989, 0.00580198,
       0.00810698, 0.00612573, 0.00739794, 0.00692249, 0.01098778,
       0.00696643, 0.00924469, 0.01107591, 0.00725217, 0.00624486,
       0.00647368, 0.0070571 , 0.00704838, 0.00518785, 0.00811477,
       0.00842399, 0.00825685, 0.00747569, 0.01499757, 0.01252349,
       0.00697467, 0.00992057, 0.00742038, 0.00794905, 0.00723046,
       0.00632363, 0.00501063, 0.00657219, 0.00534305, 0.00506221,
       0.00688314, 0.00673747, 0.00636222, 0.00605884, 0.00665261,
       0.00837939, 0.00726432, 0.0085797 , 0.01068756, 0.00868819,
       0.00784821, 0.00809191, 0.00920432, 0.00971311, 0.00541595,
       0.00683986, 0.00757175, 0.00641539, 0.0070447 , 0.00736875,
       0.00690248, 0.00666859, 0.0095858 , 0.00622759, 0.00759

In [19]:
df1=df.iloc[int(len(df)*0.8):]

In [20]:
df1.head()

,AMT,COP,CVX,AXP,BAC,C,CB,CME,GS,JPM,...,TMO,UNH,AMZN,HD,LOW,MCD,NKE,SBUX,TGT,TJX
Date,,,,,,,,,,,,,,,,,,,,,
2018-08-30,0.006287,0.011492,0.007431,0.006722,0.007536,0.008490,0.006935,0.007492,0.007423,0.007046,...,0.006108,0.005602,0.013537,0.006050,0.007002,0.004273,0.009057,0.007809,0.007861,0.007815
2018-08-31,0.005705,0.007963,0.007512,0.005785,0.007837,0.006637,0.005348,0.006729,0.008200,0.007086,...,0.005431,0.005862,0.006979,0.005940,0.008331,0.005731,0.006668,0.009329,0.005829,0.007593
2018-09-04,0.006892,0.009108,0.007998,0.007652,0.008395,0.008948,0.006127,0.007719,0.010071,0.007230,...,0.009403,0.005733,0.012059,0.009344,0.008520,0.006153,0.015962,0.006645,0.007436,0.008866
2018-09-05,0.009738,0.011318,0.007663,0.012087,0.007560,0.009352,0.007273,0.011543,0.009268,0.007624,...,0.007096,0.005926,0.014656,0.011826,0.010920,0.006066,0.012123,0.008004,0.012340,0.009471
2018-09-06,0.007024,0.011511,0.011971,0.008785,0.007499,0.009558,0.006942,0.009843,0.009062,0.007023,...,0.007207,0.006269,0.018004,0.008278,0.008295,0.005347,0.011046,0.008170,0.009716,0.010206


In [21]:
for i in range(len(df1.columns)):
  df1[df1.columns[i]]=forecasts[:,i]

<ipython-input-21-8a232fbfd1e1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[df1.columns[i]]=forecasts[:,i]


In [22]:
df1.to_excel("HAR Forecasts.xlsx")

In [23]:
test=df.iloc[int(len(df)*0.8):]
test=np.array(test)
RMSE=np.sqrt(np.sum((test-forecasts)**2)/(test.shape[0]*test.shape[1]))
MAE=(np.sum(abs(test-forecasts))/(test.shape[0]*test.shape[1]))
QLIKE=np.sum(test/forecasts-np.log(test/forecasts)-1)/(test.shape[0]*test.shape[1])
MAPE=np.sum(abs(test-forecasts)/(test))/(test.shape[0]*test.shape[1])
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

RMSE: 0.005257868312729301 | QLIKE: nan | MAE: 0.003380167069839713 | MAPE: 0.2432847852439869


<ipython-input-23-14b8c4a768c6>:5: RuntimeWarning: divide by zero encountered in divide
  QLIKE=np.sum(test/forecasts-np.log(test/forecasts)-1)/(test.shape[0]*test.shape[1])
<ipython-input-23-14b8c4a768c6>:5: RuntimeWarning: invalid value encountered in subtract
  QLIKE=np.sum(test/forecasts-np.log(test/forecasts)-1)/(test.shape[0]*test.shape[1])
